In [1]:
%matplotlib inline
import copy

import numpy as np
import torch
from torch.nn import CrossEntropyLoss
import matplotlib.pyplot as plt
cmap = plt.get_cmap("viridis")
from tqdm import tqdm

import config
from graph_models import FullGraphModel
from complete_training_data_processing import CompleteModelsDataProcessor
from utils import (
    clean_model_outputs, 
    get_image_paths, 
    get_iteration_number, 
    initialize_results_df, 
    select_random_images, 
    update_results_df, 
    update_running_loss, 
    update_config_with_sweep
)
from plots import guess_your_plots, plot_results

device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)
dtype = torch.float32
batch_size = config.batch_size

In [2]:
u_config = update_config_with_sweep(config, None)
training_images = get_image_paths(u_config.TRAINING_DATA_DIR, u_config.small_length)
data_processor = CompleteModelsDataProcessor(u_config)
model = FullGraphModel(data_processor, u_config).to(u_config.DEVICE)
criterion = CrossEntropyLoss()

In [3]:
def correct_test_results(test_results):
    # There was a bug in how we get the classes, and the 0-1 labels can be flipped
    # This function corrects the labels if the accuracy is below 0.5
    flipped = False
    if test_results["Is correct"].sum() / len(test_results) < 0.5:
        test_results["Is correct"] = np.abs(test_results["Is correct"] - 1)
        flipped = True

    return test_results, flipped

# test
def test(model):
    testing_images = get_image_paths(
        config.TESTING_DATA_DIR, config.small_length
    )
    already_selected_testing = []
    total_correct, total, running_loss = 0, 0, 0.0
    test_results = initialize_results_df()

    iterations = get_iteration_number(len(testing_images), batch_size)
    for _ in tqdm(range(iterations)):
        batch_files, already_selected_testing = select_random_images(
            testing_images, batch_size, already_selected_testing
        )
        images, labels = data_processor.get_data_from_paths(batch_files)
        inputs, labels = data_processor.process_batch(images, labels)
        inputs = inputs.to(config.DEVICE)

        out = model(inputs)
        loss = criterion(out, labels)

        # Calculate run parameters
        outputs, predictions, labels_cpu, correct = clean_model_outputs(out, labels)
        test_results = update_results_df(
            test_results, batch_files, outputs, predictions, labels_cpu, correct
        )
        running_loss += update_running_loss(loss, inputs)
        total += batch_size
        total_correct += correct.sum()

    test_results, flipped = correct_test_results(test_results)
    plot_types = (
        guess_your_plots(u_config)
        if len(u_config.plot_types) == 0
        else u_config.plot_types
    )
    final_plots = plot_results(
        test_results, plot_types=plot_types, classes=u_config.CLASSES
    )
    print(
        f"Finished testing with loss {running_loss / total} and "
        f"accuracy {total_correct / total}."
    )
    return test_results, final_plots, total_correct / total, flipped

In [4]:
checkpoint = torch.load(
    "models/model_2024-05-19 16:16:58.pth",
    map_location=device_type,
)
model.load_state_dict(checkpoint["model"])
torch.set_grad_enabled(False)
model.eval()

RuntimeError: Error(s) in loading state_dict for FullGraphModel:
	Missing key(s) in state_dict: "connectome.edge_weight". 

In [5]:
test_results, final_plots, accuracy, flipped = test(model)

100%|██████████| 37/37 [02:56<00:00,  4.76s/it]


Finished testing with loss 111420.82571396956 and accuracy 0.49155405405405406.
